In [1]:
import textract
import os
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
root = Path(".")

In [2]:
article_1 = pd.read_csv('articles1.csv')

In [3]:
news_content = article_1['content'][:500]


In [4]:
documents = list()

for file in news_content:
    ans = list()
    for i in range(len(file)):
        if file[i].isalnum():
            ans.append(file[i].lower())
        else:
            ans.append(' ')
    
    res = list()
    for i in range(len(ans) - 1):
        if ans[i] == ans[i + 1] and ans[i] == ' ':
            continue
        else:
            res.append(ans[i])
    
    if len(ans) > 0:
        res.append(ans[-1])
        documents.append(''.join(res))

In [5]:
documents

['washington congressional republicans have a new fear when it comes to their health care lawsuit against the obama administration they might win the incoming trump administration could choose to no longer defend the executive branch against the suit which challenges the administration s authority to spend billions of dollars on health insurance subsidies for and americans handing house republicans a big victory on issues but a sudden loss of the disputed subsidies could conceivably cause the health care program to implode leaving millions of people without access to health insurance before republicans have prepared a replacement that could lead to chaos in the insurance market and spur a political backlash just as republicans gain full control of the government to stave off that outcome republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the obama health care law angering conservative voters who have been demanding an end to the 

In [6]:
def get_shingle_set(documents, k):
    shingles = set()
    shingle_doc_id = dict()

    for j in range(len(documents)):
        for i in range(len(documents[j]) - k + 1):
            cur_shingle = documents[j][i : i + k]

            shingles.add(cur_shingle)
            
            if shingle_doc_id.get(cur_shingle) == None:
                shingle_doc_id[cur_shingle] = [j]
            else:
                shingle_doc_id[cur_shingle].append(j)
    
    return (shingles, shingle_doc_id)

In [7]:
for k in range(2, 11):
    print(k, len(get_shingle_set(documents, k)[0]))

2 994
3 9223
4 47147
5 149523
6 358639
7 669031
8 1040926
9 1432704
10 1802365


In [8]:
K = 4
shingles, shingle_doc_id = get_shingle_set(documents, K)
shingles = sorted(list(shingles))

In [9]:
shingles

[' 0 0',
 ' 0 1',
 ' 0 2',
 ' 0 3',
 ' 0 4',
 ' 0 5',
 ' 0 6',
 ' 0 7',
 ' 0 a',
 ' 0 b',
 ' 0 c',
 ' 0 d',
 ' 0 h',
 ' 0 m',
 ' 0 o',
 ' 0 p',
 ' 0 r',
 ' 0 t',
 ' 000',
 ' 01 ',
 ' 025',
 ' 03 ',
 ' 04 ',
 ' 046',
 ' 05 ',
 ' 053',
 ' 054',
 ' 07 ',
 ' 1 0',
 ' 1 1',
 ' 1 2',
 ' 1 3',
 ' 1 4',
 ' 1 5',
 ' 1 6',
 ' 1 7',
 ' 1 8',
 ' 1 9',
 ' 1 a',
 ' 1 b',
 ' 1 c',
 ' 1 e',
 ' 1 f',
 ' 1 g',
 ' 1 h',
 ' 1 i',
 ' 1 l',
 ' 1 m',
 ' 1 n',
 ' 1 o',
 ' 1 p',
 ' 1 r',
 ' 1 s',
 ' 1 t',
 ' 1 w',
 ' 10 ',
 ' 100',
 ' 101',
 ' 104',
 ' 105',
 ' 106',
 ' 107',
 ' 108',
 ' 109',
 ' 10t',
 ' 11 ',
 ' 110',
 ' 111',
 ' 115',
 ' 119',
 ' 11t',
 ' 12 ',
 ' 120',
 ' 121',
 ' 122',
 ' 123',
 ' 125',
 ' 126',
 ' 127',
 ' 128',
 ' 129',
 ' 12t',
 ' 13 ',
 ' 130',
 ' 131',
 ' 132',
 ' 134',
 ' 135',
 ' 138',
 ' 139',
 ' 13t',
 ' 14 ',
 ' 140',
 ' 142',
 ' 143',
 ' 145',
 ' 146',
 ' 148',
 ' 14t',
 ' 15 ',
 ' 150',
 ' 151',
 ' 152',
 ' 153',
 ' 154',
 ' 156',
 ' 157',
 ' 158',
 ' 159',
 ' 15t',
 ' 16 ',
 

In [10]:
incident_matrix = np.zeros(shape=(len(shingles), len(documents)))
incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
shingle_id = dict()
for i in range(len(shingles)):
    shingle_id[shingles[i]] = i

for shingle, doc_ids in shingle_doc_id.items():
    for doc_id in doc_ids:
        incident_matrix[shingle_id[shingle]][doc_id] = 1

incident_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
my_path = root / "Pickled_files" / "Incident_Matrix"
dbfile = open(my_path, 'wb')
pickle.dump(incident_matrix, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingles"
dbfile = open(my_path, 'wb')
pickle.dump(shingles, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Shingle_id"
dbfile = open(my_path, 'wb')
pickle.dump(shingle_id, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "documents"
dbfile = open(my_path, 'wb')
pickle.dump(documents, dbfile) 
dbfile.close()

In [15]:
for i in range(len(documents)) : 
    matrix = incident_matrix[0 : len(incident_matrix) , i]
    print(np.count_nonzero(matrix == 1) , matrix.shape[0])

2447 47147
6912 47147
5040 47147
5578 47147
1974 47147
643 47147
1916 47147
5431 47147
3995 47147
2438 47147
3101 47147
4121 47147
3733 47147
1950 47147
3920 47147
3617 47147
3767 47147
3637 47147
1913 47147
1157 47147
2503 47147
2142 47147
3394 47147
4003 47147
3543 47147
2685 47147
3531 47147
4069 47147
2518 47147
5349 47147
2803 47147
3176 47147
2517 47147
1484 47147
1726 47147
2328 47147
3667 47147
3102 47147
2130 47147
3315 47147
3309 47147
3381 47147
3546 47147
3743 47147
1822 47147
3428 47147
2456 47147
1335 47147
3259 47147
3041 47147
4107 47147
3399 47147
1344 47147
2366 47147
3764 47147
4092 47147
2147 47147
2883 47147
2800 47147
2589 47147
2116 47147
3142 47147
3663 47147
3522 47147
2910 47147
7352 47147
1868 47147
2189 47147
2074 47147
2234 47147
2229 47147
8978 47147
2971 47147
2928 47147
2469 47147
2615 47147
1254 47147
1434 47147
3264 47147
1185 47147
2772 47147
1256 47147
2017 47147
2901 47147
3547 47147
1991 47147
2573 47147
3301 47147
2624 47147
8834 47147
3367 47147
